# BALTO Graphical User Interface (A Prototype)

Scott Dale Peckham<sup>1</sup>, Maria Stoica<sup>1</sup>, D. Sarah Stamps<sup>2</sup>, James Gallagher<sup>3</sup>, Nathan Potter<sup>3</sup>, David Fulker<sup>3</sup> <br>
<sup>1</sup>University of Colorado, Boulder, United States of America;
<sup>2</sup>Virginia Tech;
<sup>1</sup>OPeNDAP, Inc.<br>
Corresponding author:  Scott.Peckham@colorado.edu

#### Table of Contents

1.  <a href="#intro">Introduction</a> <br>
2.  <a href="#start_gui">Import and Start the BALTO GUI</a> <br>
3.  <a href="#get_data">Download and Explore Some Data</a> <br>
4.  <a href="#plot_data">Plot Some Data</a> <br>
5.  <a href="#var_match">Matching Variable Names to Standardized Names</a> <br>
6.  <a href="#low_level">Low-level Access to GUI Settings</a> <br>
7.  <a href="#references">References for More Info</a> <br>
8.  <a href="#setup">Appendix 1: Set up a conda environment called "balto"</a> <br>

## Introduction  <a id="intro"></a>

This Jupyter notebook creates a GUI (graphical user interface) for the BALTO (Brokered Alignment of Long-Tail Observations) project.  BALTO is funded by the NSF EarthCube program.  The GUI aims to provide a simplified and customizable method for users to access data sets of interest on servers that support the OpenDAP data access protocol.  This interactive GUI runs within the Jupyter notebook and uses the Python packages: <b>ipywidgets</b> (for widget controls), <b>ipyleaflet</b> (for interactive maps) and <b>pydap</b> (an OpenDAP client).

The Python source code to create the GUI and to process events is in a Python module called <b>balto_gui.py</b> that must be copied into the same directory as this Jupyter notebook.  A Python module to create plots and color images from data, called <b>balto_plot.py</b>, is included as well.

This GUI can be toggled between an <b>accordion style</b> and a <b>tab style</b>.  Both styles allow you to switch between GUI panels without scrolling in the notebook.

You can run this notebook in a browser window without installing anything on your computer, using something called <b>Binder</b>.  On GitHub, on the README page, you will see a Binder icon and a link labeled "Launch Binder".

To run this Jupyter notebook without Binder, it is recommended to install Python 3.7 from an Anaconda distribution and to then create a <b>conda environment</b> called <b>balto</b>.  Instructions for how to create a conda environment are given in Appendix 1.

This GUI makes it much easier to browse, download and analyze data from servers that support the OpenDAP protocol.  Under the hood (in balto_gui.py), it performs many calculations automatically, by making use of any metadata that is available for a chosen dataset. Keep in mind, however, that this is a work in progress that will continue to be improved.

The default <b>OpenDAP URL Dir</b> in the GUI is a test server with a wide variety of datasets for testing common, unusual and edge cases.

## Import and Start the BALTO GUI  <a id="start_gui"></a>

In [2]:
import balto_gui as bg
balto = bg.balto_gui()
## balto.show_gui(ACC_STYLE=True)    # Use accordion style
balto.show_gui()   # Use tab style

Output()

## Download and Explore Some Data  <a id="get_data"></a>

First open the <b>Browse Data</b> panel.
If you changed the "OpenDAP URL Dir", click the Reset button at the bottom of the Data panel to restore the defaults.  The OpenDAP URL Dir should again be: http://test.opendap.org/dap/data/nc/.

Click on the Go button. <br>
From the <b>Filename droplist</b>, choose <b>sst.mnmean.nc.gz</b>. <br>
From the <b>Variable</b> droplist, choose <b>sst</b>.

Notice that the shape of this dataset is <b>(1857, 89, 180)</b>,
with dimensions: <b>('time', 'lat', 'lon')</b>.

Next, open the <b>Spatial Extent</b> panel and use the interactive map to choose a geographic bounding box for some region you are interested in.  Try choosing different basemaps from the Basemap droplist above the map.  You can also click on the Full Screen icon in the upper right corner to expand the map to the full screen of your computer.

You can also type new bounding lats and lons into the text boxes and click the <b>Update</b> button.  The map will then zoom to show a bounding box that contains your bounding box.  Note, however, that the aspect ratio of the GUI's map window may not match the shape of your bounding box.  (Sometimes you have to click the <b>Reset</b> button and retry to get it to work properly.)

Now let's zoom into the Carribean island of Puerto Rico.  In the text boxes, enter <b>-68.2</b> for the West edge, <b>-64.7</b> for the East edge, <b>18.9</b> for the North edge and <b>17.6</b> for the South edge, then click the <b>Update</b> button.  This should zoom in to a view in which the whole island is visible in the window, with water visible all around. You can use the "minus" button (upper left) to zoom out by one click to download a larger array.  

Let's now check how the monthly mean sea surface temperature has changed over the years spanned by the SST dataset we downloaded.  To see the date and time range spanned by this dataset, open the <b>Date Range</b> panel.  If you want, you can change the dates and times to restrict the data to a shorter time period.

Next, open the <b>Download Data</b> panel and click on the <b>Download</b> button.  This causes the SST dataset --- restricted to this spatial extent and date range --- to be downloaded to your computer and stored in the <b>balto</b> object as <b>user_var</b>.  This may take a minute or so, before you can execute the next cell.

You can now begin to work with the data in this notebook.


In [5]:
sst = balto.user_var
print('type(sst) =', type(sst))
print('sst.dtype =', sst.dtype)
print('sst.shape =', sst.shape)
print('sst.min() =', sst.min())
print('sst.max() =', sst.max())

type(sst) = <class 'numpy.ndarray'>
sst.dtype = float64
sst.shape = (2030, 1354)
sst.min() = 109.31499760437
sst.max() = 266.7999941397


Recall from the  Browse Data panel that the SST dataset has dimensions <b>('time', 'lat', 'lon')</b>.  When you chose a new bounding box and time interval in the <b>Spatial Extent</b> and <b>Date Range</b> panels of the GUI, you restricted the time, lat and lon indices, resulting in a much smaller array, as seen by the new shape.  This also resulted in a much faster download.

As can be seen in the <b>Browse Data</b> panel, the original SST data values were stored as 2-byte, signed integers.  However, looking at the Attributes in that panel, we see that there is a <b>scale_factor</b> attribute that we are supposed to multiply these integer values by in order to convert them to actual temperatures, with units of <b>degrees C</b>.  Some data sets also have an <b>add_offset</b> attribute.  The GUI looks for these, and if found, applies them for you.  This converts the data type from 2-byte, signed integer to a floating-point number.

The GUI also retrieves and stores the restricted range of times, lats and lons associated with this variable (sea surface temperature).  You can get these as follows.

In [6]:
times = balto.user_var_times
lats  = balto.user_var_lats
lons  = balto.user_var_lons
print('min(times), max(times) =', times.min(), times.max())
print('min(lats),  max(lats)  =', lats.min(), lats.max())
print('min(lons),  max(lons)  =', lons.min(), lons.max())

min(times), max(times) = 366.0 7670.85
min(lats),  max(lats)  = 17.0 19.0
min(lons),  max(lons)  = -71.0 -63.0


Note that the latitudes have the wrong sign, but that is how they were stored in the original dataset.  If you choose <b>lat</b> from the <b>Variable</b> droplist, and then look at its attributes in the <b>Attributes</b> droplist, you see that the <b>actual_range</b> is <b>[88.0, -88.0]</b>, where min and max are reversed from what you would expect.

You can do a variant of this exercise for comparison by choosing <b>coads_climatology2.nc</b> from the <b>Filename</b> droplist, choosing <b>SST</b> from the <b>Variable</b> droplist and repeating all of the other steps.  You can do another variant by choosing the filename from the <b>Filename</b> droplist that begins with <b>20070917-MODIS</b> and choosing <b>sea_surface_temperature</b> from the <b>Variable</b> droplist.

## Plot Some Data  <a id="plot_data"></a>

We can now examine how the sea surface temperatures around Puerto Rico have changed over the years.  Remember that the first dimension of the SST variable is time, and temperature values are monthly averages over a given grid cell.

First, we will plot the monthly mean SST for a single grid cell, over the full range of times.  Here we are using the <b>plot_data</b> function defined in <b>balto_plot.py</b>.

In [ ]:
import balto_plot as bp
### Plot of Monthly Mean Sea Surface Temperature vs. Time



Next, we will create a color image that shows the monthly mean SST grid for a single time.  In this case we are using the <b>show_grid</b> function defined in <b>balto_gui.py</b>, that calls a function in <b>balto_plot.py</b>.

In [ ]:
# grid = sst[10,:,:]
# balto.show_grid( grid, cmap='rainbow', xsize=8, ysize=8)

In [ ]:
### Color Image of Monthly Mean Sea Surface Temperature for a Given Time



## Matching Variable Names to Standardized Names  <a id="var_match"></a>

The [Scientific Variables Ontology](http://www.geoscienceontology.org) (SVO), developed by Peckham and Stoica (2014, 2018, 2019) provides a machine-readable mechanism for the standardized, unambiguous representation of scientific variable concepts.  Developed over many years, and spanning variables across the sciences, it can be used as the "hub" in a hub-and-spoke system to map the variable names used internally by different computational models and datasets to a common representation.

As you have seen from browsing datasets in the GUI, the names of variables used in scientific datasets are not standardized, and may simply be abbreviations.  This can make it difficult and time-consuming to find variables of interest in a collection of datasets.

We have developed an experimental <b>variable name matching</b> service for SVO, that is available at: http://34.73.227.230:8000/match_phrase/.  To use it, you simply append a variable name phrase, with separate words separated by underscores, to this URL in a browser.  This service is also embedded in the BALTO GUI object as a method called <b>get_possible_svo_names()</b>.

In [3]:
balto.get_possible_svo_names('sea surface temperature', SHOW_IRI=False)


Working...
Finished.

label = sea_surface_water__temperature
rank  = 0.9

label = sea_surface_air__temperature
rank  = 0.9

label = sea_surface_water__anomaly_of_temperature
rank  = 0.875

label = sea_surface_air__reference_temperature
rank  = 0.875

label = sea_ice_surface_air__temperature
rank  = 0.857

label = sea_surface_air-vs-water__difference_of_temperature
rank  = 0.812

label = sea_surface__slope
rank  = 0.667

label = sea_surface__longitude
rank  = 0.667

label = sea_surface__latitude
rank  = 0.667

label = sea_surface__elevation
rank  = 0.667

label = sea_water__temperature
rank  = 0.625

label = sea_bottom_surface__slope
rank  = 0.625

label = sea_bottom_surface__longitude
rank  = 0.625

label = sea_bottom_surface__latitude
rank  = 0.625

label = sea_bottom_surface__elevation
rank  = 0.625



## Low-Level Access to the GUI Settings   <a id="low_level"></a>

If you have some familiarity with the Python programming language, you can browse the 3000 or so lines of source code in <b>balto_gui.py</b> and see how to access dataset information, GUI settings and class methods directly for command-line use.  Here a few simple examples.

In [ ]:
print(balto.var_short_names)
time = balto.dataset.time
print(time.actual_range)
print(time.units)
print(balto.map_minlon_box.value)

## References for More Info  <a id="references"></a>

<b>Anaconda Python Distribution</b> <br>
https://www.anaconda.com/products/individual

<b>BALTO, an EarthCube Project</b> <br>
https://www.earthcube.org/group/brokered-alignment-long-tail-observations-balto <br>
https://www.nsf.gov/awardsearch/showAward?AWD_ID=1740704 <br>
http://balto.opendap.org/opendap/

<b>Binder Project</b> <br>
https://github.com/binder-examples

<b>ipywidgets</b> Python package <br>
https://ipywidgets.readthedocs.io/en/latest/index.html <br>
https://ipywidgets.readthedocs.io/en/latest/user_guide.html

<b>ipyleaflet</b> Python package <br> 
https://ipyleaflet.readthedocs.io/en/latest/

<b>Jupyter Lab Project</b> <br>
https://jupyterlab.readthedocs.io/en/stable/getting_started/installation.html <br>
https://jupyterlab.readthedocs.io/en/stable/user/extensions.html <br>
https://ipywidgets.readthedocs.io/en/latest/user_install.html#installing-the-jupyterlab-extension<br>

<b>Jupyter Notebook Project</b> <br>
https://jupyter.org/

<b>NetCDF CF Conventions</b> <br>
http://cfconventions.org/cf-conventions/cf-conventions.html

<b>OpenDAP Data Access Protocol</b> <br>
https://www.opendap.org/ <br>
https://opendap.github.io/documentation/QuickStart.html <br>
https://www.opendap.org/support/user-documentation

Peckham, S.D. (2014) The CSDMS Standard Names:  Cross-domain naming conventions for describing process models, data sets and their associated variables, Proceedings of the 7th Intl. Congress on Env. Modelling and Software, International Environmental Modelling and Software Society (iEMSs), San Diego, CA. (Eds.  D.P. Ames, N.W.T. Quinn, A.E. Rizzoli),  Paper 12.  http://scholarsarchive.byu.edu/iemssconference/2014/Stream-A/12/.![image.png](attachment:image.png)

<b>pydap</b> Python package <br>
https://www.pydap.org/en/latest/ <br>
https://github.com/pydap/pydap

<b>Scientific Variables Ontology</b> (SVO) <br>
http://www.geoscienceontology.org/ <br>

Stoica, M. and S.D. Peckham (2019) The Scientific Variables Ontology:  A blueprint for custom manual and automated creation and alignment of machine-interpretable qualitative and quantitative variable concepts, MWS 2019: Modeling the World’s Systems, http://pittmodelingconference.sci.pitt.edu.![image.png](attachment:image.png)

Stoica, M. and S.D. Peckham (2019) Incorporating new concepts into the Scientific Variables Ontology, Workshop on Advanced Knowledge Technologies for Science in a FAIR World, San Diego, CA, (Sept. 24, 2019) http://mint-project.info/assets/publications/stoica-peckham_escience19_abstract.pdf, https://www.isi.edu/ikcap/akts/akts2019/.![image.png](attachment:image.png)

Stoica, M. and S.D. Peckham (2018) An ontology blueprint for constructing qualitative and quantitative scientific variables, ISWC 2018: The 17th International Semantic Web Conference, Monterey, CA (Oct. 8-12), http://ceur-ws.org/Vol-2180/paper-64.pdf![image.png](attachment:image.png)

<b>traitlets</b> Python package (used by ipywidgets) <br>
https://traitlets.readthedocs.io/en/stable/

<b>appmode</b>, a Jupyter plugin (not used) <br>
https://github.com/oschuett/appmode <br>
https://github.com/binder-examples/appmode


## Appendix 1:  Set up a conda environment called "balto"    <a id="setup"></a>

To run this Jupyter notebook, it is recommended to use Python 3.7 from an Anaconda distribution and to create a "balto" conda environment with the following commands.  This helps to prevent conflicts with other Python packages you may have installed.  After installing Anaconda, use these commands to create the environment. 

``` bash
% conda update -n base -c defaults conda
% conda create --name balto
% conda activate balto
% conda install -c conda-forge ipywidgets
% conda install -c conda-forge ipyleaflet
% conda install -c conda-forge pydap
```

If you want to use this notebook in Jupyter Lab (vs. Jupyter Notebook), you need to install a few more Python packages and extensions, as follows.

``` bash
% conda install -c conda-forge jupyterlab
% conda install -c conda-forge nb_conda_kernels  # (needed for conda envs)

% conda install -c conda-forge nodejs
% conda install -c conda-forge widgetsnbextension
% jupyter labextension install jupyter-leaflet
% jupyter labextension install @jupyter-widgets/jupyterlab-manager
```

When you want to run the notebook, type "conda activate balto" to activate this environment.  Then change to the directory that contains BALTO_GUI.ipynb and type "jupyter notebook" or "jupyter lab".

Finally, choose this notebook by name, "BALTO_GUI.ipynb", and make sure to choose the kernel called:  Python [conda env:balto].  See the References section at the end for more info.